In [1]:
import numpy  as np


pos = np.load(r'/media/ncclab306/database7/wangm/simnibs_leadfield/pos_hcp2_2.npy')
lfm = np.load(r'/media/ncclab306/database7/wangm/simnibs_leadfield/lfm_hcp2_2.npy')


In [3]:
model = '22_2_second'
label ='hippo'
TARGET_POSITION = [-31,-20,-14] # dlpfc  [-39, 34, 37] # hippo [-31,-20,-14]
position = TARGET_POSITION
distance = np.zeros(len(pos))
for i in range(len(pos)):
    distance[i] = (pos[i, 0] - position[0])**2 + (pos[i, 1] - position[1])**2 + (pos[i, 2] - position[2])**2

AVOID_POSITION = np.where(distance < 10**2)
AVOID_POSITION = AVOID_POSITION[0]
print(len(AVOID_POSITION))
print(len(AVOID_POSITION)/len(pos))


NUM_ELE = 75
def tis_function6_focality(x):
    #print(x)
    eam = np.zeros(len(pos))
    electrode1 = int(round(x[2] * 74))
    electrode2 = int(round(x[3] * 74))
    stimulation1 = np.zeros(NUM_ELE)
    stimulation1[electrode1] = 2 * x[0]
    stimulation1[electrode2] = -(2 * x[0])
    e1 = np.array([np.matmul(lfm[:, :, 0].T, stimulation1), np.matmul(lfm[:, :, 1].T, stimulation1),np.matmul(lfm[:, :, 2].T, stimulation1)]).T /1000
    #print(np.max(e1))
    electrode3 = int(round(x[4] * 74))
    electrode4 = int(round(x[5] * 74))
    print(electrode1,electrode2,electrode3,electrode4,x[0],x[1])
    stimulation2 = np.zeros(NUM_ELE)
    # stimulation2[electrode3] = 1.5 - x[0]
    # stimulation2[electrode4] = -1.5 + x[0]
    stimulation2[electrode3] = 2 * x[1]
    stimulation2[electrode4] = -(2 * x[1])
    e2 = np.array([np.matmul(lfm[:, :, 0].T, stimulation2), np.matmul(lfm[:, :, 1].T, stimulation2),np.matmul(lfm[:, :, 2].T, stimulation2)]).T / 1000
    for i in range(len(e1)):
        l_x = np.sqrt(e1[i].dot(e1[i]))
        l_y = np.sqrt(e2[i].dot(e2[i]))
        point = e1[i].dot(e2[i])
        cos_ = point/(l_x*l_y)
        #print(cos_)
        if cos_ <= 0:   # one of the fields must be flipped when a > 90 degrees
            e1[i] = -e1[i]
            cos_ = -cos_
        if l_y < l_x:
            if l_y < l_x * cos_:
                eam[i] = 2 * l_y
            else:
                eam[i] = 2 * np.linalg.norm(np.cross(e2[i], (e1[i] - e2[i]))) / np.linalg.norm(e1[i] - e2[i])
        else:
            if l_x < l_y * cos_:
                eam[i] = 2 * l_x
            else:

                eam[i] = 2 * np.linalg.norm(np.cross(e1[i], (e2[i] - e1[i]))) / np.linalg.norm(e2[i] - e1[i])
    return eam[AVOID_POSITION],eam

x = np.array([
4.265612614755935894e-01,4.771678331211757218e-01,9.837044977606771434e-01,2.695317390691184256e-01,9.706700989450416373e-01,2.834544126524854990e-01

])

a,b = tis_function6_focality(x)

np.save("checklzm/all_ti_"+model+label+".npy",b)
np.save("checklzm/target_ti"+model+label+".npy",a)
print("done")


def tdcs_function_vol(s):
    eam = ((np.matmul(lfm[:, :, 0].T, s)) ** 2 + (np.matmul(lfm[:, :, 1].T, s)) ** 2 + (
        np.matmul(lfm[:, :, 2].T, s)) ** 2) ** 0.5
    return eam[AVOID_POSITION]/1000,eam/1000

x = np.array([

2.783860705079075834e-03,-5.618109135110500515e-03,3.160932336743794285e-03,-1.095163998530400418e-04,-4.688172627193551434e-03,-4.879208723267282333e-03,-5.292600010216357009e-03,5.242845340473960974e-03,1.122630514545627789e-03,9.311393291996850063e-01,-5.773076576827677886e-03,7.253368917213540851e-03,3.399445203409443006e-03,3.603546279431210815e-03,4.573157356037849813e-03,-3.911613030344936182e-03,4.191571006636772567e-03,6.973836587194637282e-04,-1.997446239941295942e-03,3.686660490207299690e-03,9.865801487379194379e-01,-1.398524561685841883e-02,4.804701994302381513e-04,1.272580579831124672e-03,2.473201547116443688e-03,-8.127248107039596048e-04,-2.955667578117670419e-03,3.012740476200752304e-03,2.018640122846714936e-03,-1.167918846355267548e-03,4.228491394136370263e-03,1.540170899655512841e-03,-1.664627818529110931e-03,4.281619126360099864e-03,-5.567139258566403126e-03,3.508638851102618625e-03,-4.294989921904044348e-03,1.677097086775328899e-03,1.217129516845061495e-03,-3.324586676043714967e-03,-5.367682242562194644e-04,3.250254159588440717e-03,3.146738669259865895e-03,1.163625600351091485e-03,2.857931530757296806e-03,1.935622171245737766e-03,-2.123242865356336079e-03,6.266165828386245640e-04,2.839539744398319326e-03,3.552266999052006158e-03,-4.125665171350927085e-03,-3.598408570859176758e-03,-6.204109542131379038e-03,-1.242914709310198595e-03,-3.894322693088977244e-03,-8.327307038753915813e-03,3.214576231464153638e-04,-1.623434475688793899e-03,-6.398299356883575296e-03,1.864985055647271073e-03,6.132050826283860778e-03,-2.590335602659088465e-05,-8.840565673983898087e-04,-1.217705702841385268e-03,4.051238806923887492e-03,-4.517019235580387385e-03,-1.181871176730946160e-02,7.250414313726326332e-03,-1.006862682512014301e-02,8.048551000311189585e-04,-5.775424399289269539e-04,1.660233130751573305e-03,-9.342980044138151641e-01,3.840448237457008696e-03,-9.301114148746884025e-01

])

a,b = tdcs_function_vol(x)

np.save("checklzm/all_tacs_"+model+label+".npy",b)
np.save("checklzm/target_tacs_"+model+label+".npy",a)
print("done")


4621
0.0024999431954045625
73 20 72 21 0.4265612614755936 0.4771678331211757
done
done
